# Hora de Código
## Desarrollando 

Supongamos nos encomiendan la tarea de escribir una sequela para el Libro Don Quixote de la Mancha

* **Análisis**
* **Modelación**
* **Producción**

In [2]:
import re
import pickle
import PyPDF2
import requests
import numpy as np
from io import BytesIO
from unidecode import unidecode
from collections import deque
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [19]:
from collections import Counter

## Primeros Pasos

In [ ]:
url = "http://www.gutenberg.org/cache/epub/2000/pg2000.txt"
r = requests.get(url)

In [83]:
corpus = r.text
init_book = corpus.find("En un lugar de la Mancha")
end_book = corpus.find("End of Project Gutenberg's")
corpus = corpus[init_book: end_book]
corpus = unidecode(corpus.replace("\r\n", " "))
corpus = re.sub("[^\w\s]", "", corpus)

In [84]:
corpus[:500]

'En un lugar de la Mancha de cuyo nombre no quiero acordarme no ha mucho tiempo que vivia un hidalgo de los de lanza en astillero adarga antigua rocin flaco y galgo corredor Una olla de algo mas vaca que carnero salpicon las mas noches duelos y quebrantos los sabados lantejas los viernes algun palomino de anadidura los domingos consumian las tres partes de su hacienda El resto della concluian sayo de velarte calzas de velludo para las fiestas con sus pantuflos de lo mesmo y los dias de entreseman'

In [85]:
corpus[:500]

'En un lugar de la Mancha de cuyo nombre no quiero acordarme no ha mucho tiempo que vivia un hidalgo de los de lanza en astillero adarga antigua rocin flaco y galgo corredor Una olla de algo mas vaca que carnero salpicon las mas noches duelos y quebrantos los sabados lantejas los viernes algun palomino de anadidura los domingos consumian las tres partes de su hacienda El resto della concluian sayo de velarte calzas de velludo para las fiestas con sus pantuflos de lo mesmo y los dias de entreseman'

In [86]:
# Cleaning Text
text = corpus.lower()
#text = clean_text(text)
tokens = text.split()

In [87]:
tokens[:10]

['en', 'un', 'lugar', 'de', 'la', 'mancha', 'de', 'cuyo', 'nombre', 'no']

We now look to make a sequence of characters

In [88]:
word_counter = Counter(tokens)
word_counter.most_common(10)

[('que', 21246),
 ('de', 18031),
 ('y', 17980),
 ('la', 10224),
 ('a', 9718),
 ('el', 9346),
 ('en', 8109),
 ('no', 6275),
 ('se', 5025),
 ('los', 4701)]

In [6]:
# Total number of sequences inside the dataset 
nseq = len(sequences); nseq

9071

In [89]:
elements = []
for w0, w1 in zip(tokens[0:-1], tokens[1:len(tokens)]):
    element = (w0, w1)
    elements.append(element)

In [90]:
bigrams = Counter(elements)
bigrams.most_common(10)

[(('don', 'quijote'), 2149),
 (('de', 'la'), 2052),
 (('lo', 'que'), 1535),
 (('que', 'no'), 1284),
 (('en', 'el'), 1042),
 (('de', 'los'), 965),
 (('que', 'se'), 939),
 (('en', 'la'), 934),
 (('de', 'su'), 900),
 (('a', 'la'), 900)]

Hasta ahora tenemos lo _bigram_ más comúnes, ¿de qué manera podemos conocer los _bigrams_ que empiecen con ciertas palabras?

In [130]:
topv = sorted(filter(lambda w: w[0] == "don", bigrams),
                     key=lambda w: bigrams[w])[:-5:-1]
for v in topv:
    print(v, bigrams[v])

('don', 'quijote') 2149
('don', 'fernando') 131
('don', 'antonio') 62
('don', 'luis') 36


In [ ]:
elements = []
for w0, w1 in zip(tokens[0:-1], tokens[1:len(tokens)]):
    element = (w0, w1)
    elements.append(element)